In [36]:
## Sebastian Puerta-Hincapie
## Assignment #5 - Integration
## CSC 301 - Numerical Issues
## Prof. Grimmelmann

# Libraries
import math;
import time;
import numpy;
from scipy.integrate import quad
from prettytable import PrettyTable

# global variables
N = 20
# Radian values
UPPER = 1.6
LOWER = 0.4

# table set up
display_table =  PrettyTable()
display_table.field_names = ['Exact','Measured','Absolute','Relative','N']

# functions
# ACTUAL (EXPECTED) INTEGRATION RESULT
def actual_integration1():
    Exact = -math.cos(UPPER)+math.cos(LOWER)+math.sin(UPPER)-math.sin(LOWER)
    #Exact = -math.cos(UPPER_DEG)+math.cos(LOWER_DEG)+math.sin(UPPER_DEG)-math.sin(LOWER_DEG)
    return Exact

def func(x):
    return math.sin(x)+math.cos(x)

def actual_integration2():
    Exact , Error = quad(func,LOWER,UPPER)
    return Exact

# MEASURED INTEGRATION RESULT

# Numerical Quadrature
def method_one():
    # initializing
    display_table.clear_rows()
    
    for n in range(1,N+1):
        # function
        ABSOLUTE = RELATIVE = RESULT = tmp = 0
        for i in range(1,n+1):
            tmp += (math.sin(LOWER + ((i - 0.5)) * ((UPPER - LOWER) / n)) + math.cos(LOWER + ((i - 0.5)) * ((UPPER - LOWER) / n)))
        RESULT = ((UPPER - LOWER) / n) * tmp

        # error
        ABSOLUTE = abs(RESULT - actual_integration1())
        RELATIVE = ABSOLUTE / actual_integration1()

        # display
        display_table.add_row([actual_integration1(),RESULT,ABSOLUTE,RELATIVE,n])
        
    print(display_table)

# Trapezoidal rule
def method_two():
    N = 150
    # initializing
    display_table.clear_rows()
    
    for n in range(130,N+1):
        ABSOLUTE = RELATIVE = RESULT = tmp = 0
        # function
        h = (UPPER - LOWER) / n
        tmp = 0.5*(math.sin(UPPER) + math.cos(UPPER)) + 0.5*(math.sin(LOWER) + math.cos(LOWER))
        for i in range(1,n+1):
            tmp += (math.sin(LOWER + i * h) + math.cos(LOWER + i * h))
        RESULT = h * tmp

        # error
        ABSOLUTE = abs(RESULT - actual_integration1())
        RELATIVE = ABSOLUTE / actual_integration1()

        # display
        display_table.add_row([actual_integration1(),RESULT,ABSOLUTE,RELATIVE,n])
        
    print(display_table)
    
# Simpson's Rule
def method_three():
    # initializing
    display_table.clear_rows()
    
    for n in range(1,N+1):
        ABSOLUTE = RELATIVE = RESULT = tmp =  0
        # function
        h = (UPPER - LOWER) / n
        p = LOWER + h
        for i in range(1,int(n*0.5)+1):
            tmp += 4 * (math.sin(p) + math.cos(p))
            p += 2 * h
        p = LOWER + (2*h)
        for i in range(1,int(n*0.5)):
            tmp += 2 * (math.sin(p) + math.cos(p))
            p += 2 * h
        RESULT = (h/3) * (math.sin(UPPER) + math.cos(UPPER) + math.sin(LOWER) + math.cos(LOWER) + tmp)

        # error
        ABSOLUTE = abs(RESULT - actual_integration1())
        RELATIVE = ABSOLUTE / actual_integration1()

        # display
        display_table.add_row([actual_integration1(),RESULT,ABSOLUTE,RELATIVE,n])
        
    print(display_table)

# Romberg Integration
def method_four():
    # initializing
    display_table.clear_rows()
    
    for n in range(1,N+1):
        # function
        ABSOLUTE = RELATIVE = RESULT = tmp = 0
        arr = numpy.array( [[0] * (n+1)] * (n+1), float )
        h = UPPER - LOWER
        arr[0,0] = 0.5 * h * ( math.sin(LOWER) + math.cos(LOWER) + math.sin(UPPER) + math.cos(UPPER) )

        powerOf2 = 1
        for i in range( 1, n + 1 ):
            # Compute the halved stepsize and use this to sum the function at
            # all the new points (in between the points already computed)
            
            h = 0.5 * h
            tmp = 0.0
            powerOf2 = 2 * powerOf2
            for k in range( 1, powerOf2, 2 ):
                tmp = tmp + math.sin( LOWER + k * h ) + math.cos( LOWER + k * h )

            # Compute the composite trapezoid rule for the next level of
            # subdivision.  Use Richardson extrapolation to refine these values
            # into a more accurate form.

            arr[i,0] = 0.5 * arr[i-1,0] + tmp * h

            powerOf4 = 1
            for j in range( 1, i + 1 ):
                powerOf4 = 4 * powerOf4
                arr[i,j] = arr[i,j-1] + ( arr[i,j-1] - arr[i-1,j-1] ) / ( powerOf4 - 1 )

        RESULT = arr[n-1,n-1]

        # error
        ABSOLUTE = abs(RESULT - actual_integration1())
        RELATIVE = ABSOLUTE / actual_integration1()

        # display
        display_table.add_row([actual_integration1(),RESULT,ABSOLUTE,RELATIVE,n])
    print(display_table)

# Mid-Point
def method_five():
    # initializing
    display_table.clear_rows()
    sine = math.sin
    cosine = math.cos
    
    for n in range(1,N+1):
        ABSOLUTE = RELATIVE = RESULT = tmp =  0
        h = float(UPPER-LOWER)/n


        for i in range(n):
            tmp += (sine((LOWER + (h/2.0)) + (i * h)) + cosine((LOWER + (h/2.0)) + (i * h)))
        tmp *= h
        
        RESULT = tmp
        
        # error
        ABSOLUTE = abs(RESULT - actual_integration1())
        RELATIVE = ABSOLUTE / actual_integration1()
        
        # display
        display_table.add_row([actual_integration1(),RESULT,ABSOLUTE,RELATIVE,n])

    print(display_table)
    
# Adaptive Quadrature
def method_six():
    # initializing
    display_table.clear_rows()
    for n in range(1,N+1):
        # function
        ABSOLUTE = RELATIVE = RESULT = tmp = 0
        h = (UPPER - LOWER) / n
        #for i in range(n):
        #    f1 = math.sin(LOWER) + math.cos(LOWER)
        #    f2 = math.sin(LOWER + (h/2)) + math.cos(LOWER + (h/2))
        #    f3 = math.sin(LOWER + h) + math.cos(LOWER + h)
        #    f4 = math.sin(LOWER + ((3*h)/2)) + math.cos(LOWER + ((3*h)/2))
        #    f5 = math.sin(UPPER) + math.cos(UPPER)
        #    tmp += (((h/6)*(f1 + (4*f2) + (2*f3) + (4*f4) + f5)) - ((h/2)**4 * f1))
        #RESULT = tmp * h

        k = 0.0
        for i in range (n):
            k += ((h/6)*( \
                 (math.sin(LOWER) + math.cos(LOWER)) + \
                 (4*(math.sin(LOWER+(h/2)) + math.cos(LOWER+(h/2)))) + \
                 (2*(math.sin(LOWER + h) + math.cos(LOWER+h))) + \
                 (4*(math.sin(LOWER+(3*h/2)) + math.cos(LOWER+(3*h/2)))) + \
                 (math.sin(UPPER) + math.cos(UPPER)) \
                 ))
        k *= h
        RESULT = k
    
        # error
        ABSOLUTE = abs(RESULT - actual_integration1())
        RELATIVE = ABSOLUTE / actual_integration1()

        # display
        display_table.add_row([actual_integration1(),RESULT,ABSOLUTE,RELATIVE,n])
    print(display_table)    
    

# main program
def main():
    print("===== Start of Program =====\n")
    
    print("===== METHOD 1 (Numerical Quad) ======")
    method_one()
    print("\n===== METHOD 2 (Trapezoid) ======")
    method_two()
    print("\n===== METHOD 3 (Simpson's Rule) ======")
    method_three()
    print("\n===== METHOD 4 (Romberg Integration) ======")
    method_four()
    print("\n===== METHOD 5 (Mid-Point Formula) ======")
    method_five()
    
    #print("\n===== METHOD 6 (Adaptive Quad)======")
    #method_six()
    
    print("\n===== End of Program =====")
if __name__ == "__main__":
    main()
    
'''
Discussion:
Quadrature ~ Integration

For this assignment the methods attempted were: Numerical Quadrature, Trapezoid Rule, Simpson's Rule, 
Adaptive Quadrature, Romberg Integration, and Mid-Point. For each method I will only be displaying N = 20, 
results in order to check the consistency of the algorithm.

It was challenging to properly implement accurate algorithms for each method, since any missing part to the 
algorithm would completely throw off the end result and would give a very large error in comparison to the 
exact/expected value. Out of the six attempted methods, five gave accurate results. For the Adaptive Quadrature method, the integral result
the results were not consistent - the algorithm did not offer a stable result that was close to the exact value.
In general, the larger the N the more accurate results that were obtained. By observing the results for each method,
in order from most accurate to least accurate - (1) Romberg, (2) Simpson, (3) Mid-Point, (4) Numerical Quad, and
(5) Trapezoid.

Romberg and Simposon give the least errors and have values that are equal or very close to the exact value.
Midpoint and Numerical Quad give close results to each other and are also close to the exact value, but less so 
than the ladder methods.
Even though the Trapezoid method does eventually give very accurate results, it happens only if N > 120. That is why
in the chart I am only displaying the results when the N values range from [130,150] to show that as the N 
increases so does the accuracy.

Furthermore, it is also worth noting that for the Romberg method there is greater accuracy but in return, it is a
significantly slower algorithm. It takes a long time to run say N = >=25 iterations. However, this is to be expected
since this algorithm is running three nested for loops, which greatly reduces the performance.

The different numerical integration methods introduced performed as expected since I was able to obtain
values that are relatively close to the exact value and with low error.

- Sebastian Puerta Hincapie
'''    

===== Start of Program =====

===== METHOD 1 (Numerical Quad) ======
+--------------------+--------------------+------------------------+------------------------+----+
|       Exact        |      Measured      |        Absolute        |        Relative        | N  |
+--------------------+--------------------+------------------------+------------------------+----+
| 1.5604157770370286 | 1.6581279488112437 |   0.0977121717742151   |  0.06261931801263529   | 1  |
| 1.5604157770370286 | 1.5840701331383762 |  0.023654356101347584  |  0.015159008547236874  | 2  |
| 1.5604157770370286 | 1.5708673007477805 |  0.010451523710751864  |  0.006697909534468806  | 3  |
| 1.5604157770370286 | 1.5662827330695224 |  0.005866956032493764  | 0.0037598671577354485  | 4  |
| 1.5604157770370286 | 1.564167076065395  | 0.0037512990283663505  | 0.0024040381310995503  | 5  |
| 1.5604157770370286 | 1.5630195073430908 |  0.002603730306062202  | 0.0016686131634776565  | 6  |
| 1.5604157770370286 | 1.562328129257254

"\nDiscussion:\nQuadrature ~ Integration\n\nFor this assignment the methods attempted were: Numerical Quadrature, Trapezoid Rule, Simpson's Rule, \nAdaptive Quadrature, Romberg Integration, and Mid-Point. For each method I will only be displaying N = 20, \nresults in order to check the consistency of the algorithm.\n\nIt was challenging to properly implement accurate algorithms for each method, since any missing part to the \nalgorithm would completely throw off the end result and would give a very large error in comparison to the \nexact/expected value. Out of the six attempted methods, five gave accurate results. For the Adaptive Quadrature method, the integral \n results were not consistent - the algorithm did not offer a stable result that was close to the exact value.\nIn general, the larger the N the more accurate results that were obtained. By observing the results for each method,\nin order from most accurate to least accurate - (1) Romberg, (2) Simpson, (3) Mid-Point, (4) Nume